In [1]:
import numpy as np

In [2]:
def restricted_log_likelihood(X, Z, y, delta):
    #X_i - n_i times p - fixed effects regressor matrix
    #Z_i - n_i times q - random effects regressor matrix
    #delta - q times q - Cholesky factor of covariance matrix for random effects
    #y_i - n_i times 1 - data vector 
    M = len(X)
    N = sum([X[i].shape[0] for i in range(M)])
    p = X[0].shape[1]
    q = Z[0].shape[1]
    Z_aug = [np.vstack((Z[i], delta)) for i in range(M)] # +q rows
    X_aug = [np.vstack((X[i], np.zeros((q, p)))) for i in range(M)]
    y_aug = [np.vstack((y[i], np.zeros((q, 1)))) for i in range(M)]
    Q = []
    R11 = []
    for i in range(M):
        q0, r0 = np.linalg.qr(Z_aug[i], mode='complete')
        Q.append(q0)
        R11.append(r0[:q, :])
    R00 = [(Q[i].T @ X_aug[i])[q:, :] for i in range(M)]
    C0 = [(Q[i].T @ y_aug[i])[q:, :] for i in range(M)]
    R00_stacked = np.vstack(tuple(R00))
    C0_stacked = np.vstack(tuple(C0))
    R_C_stacked = np.hstack((R00_stacked, C0_stacked))
    r_RC = np.linalg.qr(R_C_stacked, mode='complete')[1]
    R00_real = r_RC[:p, :p]
    c_m1 = r_RC[:, -1][p:]
    res_log_l = -(N-p)*np.log(np.linalg.norm(c_m1))-np.log(np.abs(np.linalg.det(R00_real)))
    for i in range(M):
        res_log_l += np.log(np.abs(np.linalg.det(delta)/np.linalg.det(R11[i])))
    return res_log_l

In [3]:
#test1
X = [np.random.rand(3, 3), np.random.rand(3, 3)]
Z = [np.random.rand(3, 4), np.random.rand(3, 4)]
y = [np.random.rand(3, 1), np.random.rand(3, 1)]
delta = np.random.rand(4, 4)
restricted_log_likelihood(X, Z, y, delta)

-1.9072035473309548

In [4]:
#test2
p = np.random.randint(4, 6)
q = np.random.randint(4, 6)
n_0 = np.random.randint(2, 4)
n_1 = np.random.randint(2, 4)
n_2 = np.random.randint(2, 4)
X = [np.random.rand(n_0, p), np.random.rand(n_1, p), np.random.rand(n_2, p)]
Z = [np.random.rand(n_0, q), np.random.rand(n_1, q), np.random.rand(n_2, q)]
y = [np.random.rand(n_0, 1), np.random.rand(n_1, 1), np.random.rand(n_2, 1)]
delta = np.random.rand(q, q)
restricted_log_likelihood(X, Z, y, delta)

2.967822010061638